In [1]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack #this line needs to be here in case we are running the GPU version of cuda in haystack this makes it run.
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-gj3c0ges/farm-haystack_7b2644e91049462db4fc972b92dc618d
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-gj3c0ges/farm-haystack_7b2644e91049462db4fc972b92dc618d
  Resolved https://github.com/deepset-ai/haystack.git to commit a084a982c4e5a3e7b9a96ccd6bbae1a703721019
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
from haystack.nodes import DensePassageRetriever
from haystack.utils import fetch_archive_from_http
from haystack.document_stores import InMemoryDocumentStore

INFO - haystack.document_stores.base -  Numba not found, replacing njit() with no-op implementation. Enable it with 'pip install numba'.
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [2]:
from haystack.nodes import PDFToTextConverter
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.nodes import FARMReader, ElasticsearchRetriever
from haystack.pipelines import ExtractiveQAPipeline, DocumentSearchPipeline
from haystack.nodes import PreProcessor
from haystack import Document

In [3]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [4]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_1"

In [5]:
retriever = DensePassageRetriever(
    document_store=InMemoryDocumentStore(),
    query_embedding_model=query_model,
    passage_embedding_model=passage_model,
    max_seq_len_query=64,
    max_seq_len_passage=400,
    use_gpu=False
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find IIC/dpr-spanish-question_encoder-allqa-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Automatically detected language from language model name: spanish
INFO - haystack.modeling.model.language_model -  Loaded IIC/dpr-spanish-question_encoder-allqa-base
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find IIC/dpr-spanish-passage_e

In [6]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=5,
    batch_size=5,
    grad_acc_steps=3,
    save_dir=save_dir,
    evaluate_every=3,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=2,
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /|\  /w\   /|\  /w\   /w\   /w\   /w\   /|\  /w\   /w\   /w\   /w\   /w\   /w\   /w\   /|\
INFO - haystack.modeling.data_handler.data_silo -  /'\   /'\   /'\   / \   /'\   

INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8986666666666666
INFO - haystack.modeling.evaluation.eval -  f1: 0.24
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5693333333333332
INFO - haystack.modeling.evaluation.eval -  average_rank: 2.08
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9457    0.9457    0.9457       350
     positive     0.2400    0.2400    0.2400        25

     accuracy                         0.8987       375
    macro avg     0.5929    0.5929    0.5929       375
 weighted avg     0.8987    0.8987    0.8987       375

Train epoch 0/4 (Cur. train loss: 5.5996):  57%|▌| 12/21 [02:18<01:37, 10.79s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:12<00:00,  2.50s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^

Evaluating: 100%|█████████████████████████████████| 5/5 [00:12<00:00,  2.53s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 27 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.8329813003540039
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.9146666666666666
INFO - haystack.modeling.evaluation.eval -  f1: 0.36
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.6373333333333333
INFO - haystack.modeling.evaluation.eval -  average_rank: 1.56
INFO - haystack.modelin

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.6881070327758789
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.92
INFO - haystack.modeling.evaluation.eval -  f1: 0.4000000000000001
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.66
INFO - haystack.modeling.evaluation.eval -  average_rank: 1.4
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9571    0.9571    0.9571       350
     positive     0.4000    0.4000    0.4000        25

     accuracy                         0.9200       375
    macro avg     0.6786    0.6786    0.6786       375
 weighted avg     0.9200    0.9200    0.9200       375

Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.37s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//   

Train epoch 2/4 (Cur. train loss: 0.6662):  86%|▊| 18/21 [03:26<00:30, 10.07s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:12<00:00,  2.49s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 60 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.648615517616272
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.9146666666666666
INFO - haystack.modeling.evaluation.eval -  f1: 0.36
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.6373333333333333
INFO -

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.6078419494628906
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.9146666666666666
INFO - haystack.modeling.evaluation.eval -  f1: 0.36
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.6373333333333333
INFO - haystack.modeling.evaluation.eval -  average_rank: 1.36
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9543    0.9543    0.9543       350
     positive     0.3600    0.3600    0.3600        25

     accuracy                         0.9147       375
    macro avg     0.6571    0.6571    0.6571       375
 weighted avg     0.9147    0.9147    0.9147       375

Train epoch 3/4 (Cur. train loss: 0.9589):  71%|▋| 15/21 [02:50<01:00, 10.15s/it
Evaluating: 100%|█████████████████████████████████|

INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9514    0.9514    0.9514       350
     positive     0.3200    0.3200    0.3200        25

     accuracy                         0.9093       375
    macro avg     0.6357    0.6357    0.6357       375
 weighted avg     0.9093    0.9093    0.9093       375

Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.32s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 93 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_2"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=1,
    batch_size=8,
    grad_acc_steps=1,
    save_dir=save_dir,
    evaluate_every=40,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=2,
    optimizer_name = "SGD",
    learning_rate = 1e-6
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /|\  /|\  /|\  /|\  /w\   /|\  /|\  /w\   /w\   /w\   /|\  /|\  /w\   /w\   /|\  /w\ 
INFO - haystack.modeling.data_handler.data_silo -  /'\   /'\   /'\   /'\   / \   /'\  

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_3"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=1,
    batch_size=4,
    grad_acc_steps=1, #aqui
    save_dir=save_dir,
    evaluate_every=4, #aqui
    embed_title=True,
    num_positives=1,
    num_hard_negatives=1,
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /w\   /w\   /w\   /|\  /w\   /|\  /w\   /w\   /w\   /w\   /|\  /w\   /w\   /w\   /|\  /w\ 
INFO - haystack.modeling.data_handler.data_silo -  / \   / \   / \   /'\   / \   

INFO - haystack.modeling.evaluation.eval -  average_rank: 1.52
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.8876    0.8876    0.8876       169
     positive     0.2400    0.2400    0.2400        25

     accuracy                         0.8041       194
    macro avg     0.5638    0.5638    0.5638       194
 weighted avg     0.8041    0.8041    0.8041       194

Train epoch 0/0 (Cur. train loss: 3.5656):  62%|▌| 16/26 [01:33<00:52,  5.27s/it
Evaluating: 100%|█████████████████████████████████| 7/7 [00:07<00:00,  1.12s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 16 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_4"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=3,
    batch_size=6,
    grad_acc_steps=1, #aqui
    save_dir=save_dir,
    evaluate_every=4, #aqui
    embed_title=True,
    num_positives=1,
    num_hard_negatives=2,
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /w\   /w\   /w\   /|\  /w\   /|\  /w\   /w\   /w\   /w\   /w\   /w\   /w\   /|\  /w\   /w\ 
INFO - haystack.modeling.data_handler.data_silo -  / \   /'\   /'\   /'\   / \  

INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.903448275862069
INFO - haystack.modeling.evaluation.eval -  f1: 0.16
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5317241379310345
INFO - haystack.modeling.evaluation.eval -  average_rank: 2.56
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9488    0.9488    0.9488       410
     positive     0.1600    0.1600    0.1600        25

     accuracy                         0.9034       435
    macro avg     0.5544    0.5544    0.5544       435
 weighted avg     0.9034    0.9034    0.9034       435

Train epoch 0/2 (Cur. train loss: 4.0696):  94%|▉| 16/17 [03:39<00:10, 10.12s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.32s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^

Train epoch 1/2 (Cur. train loss: 2.2210): 100%|█| 17/17 [03:12<00:00, 11.35s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:12<00:00,  2.45s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 36 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.5145810365676879
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.9218390804597701
INFO - haystack.modeling.evaluation.eval -  f1: 0.32
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.620919540229885
INFO -

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_5"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=2,
    batch_size=4,
    grad_acc_steps=1,
    save_dir=save_dir,
    evaluate_every=6,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=2,
    optimizer_name = "SGD"
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /w\   /w\   /w\   /w\   /w\   /w\   /|\  /|\  /w\   /w\   /w\   /w\   /w\   /|\  /|\  /w\ 
INFO - haystack.modeling.data_handler.data_silo -  / \   / \   /'\   / \   /'\   

INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8419243986254296
INFO - haystack.modeling.evaluation.eval -  f1: 0.08
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.46096219931271476
INFO - haystack.modeling.evaluation.eval -  average_rank: 2.0
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9135    0.9135    0.9135       266
     positive     0.0800    0.0800    0.0800        25

     accuracy                         0.8419       291
    macro avg     0.4968    0.4968    0.4968       291
 weighted avg     0.8419    0.8419    0.8419       291

Train epoch 0/1 (Cur. train loss: 5.5990):  92%|▉| 24/26 [02:54<00:12,  6.34s/it
Evaluating: 100%|█████████████████████████████████| 7/7 [00:11<00:00,  1.71s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^

Train epoch 1/1 (Cur. train loss: 5.0550): 100%|█| 26/26 [03:06<00:00,  7.16s/it
Evaluating: 100%|█████████████████████████████████| 7/7 [00:12<00:00,  1.72s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | TEST SET | AFTER 52 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 2.685828237533569
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8487972508591065
INFO - haystack.modeling.evaluation.eval -  f1: 0.12
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.48439862542955325
INFO

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_6"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=2,
    batch_size=5,
    grad_acc_steps=2,
    save_dir=save_dir,
    evaluate_every=6,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=1,
    optimizer_name = "AdamW"
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /|\  /|\  /w\   /w\   /w\   /w\   /w\   /|\  /w\   /|\  /w\   /|\  /w\   /w\   /w\   /w\ 
INFO - haystack.modeling.data_handler.data_silo -  /'\   /'\   / \   / \   /'\   /

INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9200    0.9200    0.9200       225
     positive     0.2800    0.2800    0.2800        25

     accuracy                         0.8560       250
    macro avg     0.6000    0.6000    0.6000       250
 weighted avg     0.8560    0.8560    0.8560       250

Train epoch 0/1 (Cur. train loss: 12.2828): 100%|█| 21/21 [02:06<00:00,  6.04s/i
Evaluating: 100%|█████████████████████████████████| 5/5 [00:07<00:00,  1.50s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 24 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_7"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=1,
    batch_size=6,
    grad_acc_steps=1,
    save_dir=save_dir,
    evaluate_every=2,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=2,
    optimizer_name = "Adagrad"
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /w\   /w\   /|\  /w\   /w\   /|\  /|\  /w\   /w\   /w\   /|\  /w\   /|\  /w\   /|\  /w\ 
INFO - haystack.modeling.data_handler.data_silo -  /'\   /'\   /'\   / \   / \   /'

INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8942528735632184
INFO - haystack.modeling.evaluation.eval -  f1: 0.08
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.48712643678160916
INFO - haystack.modeling.evaluation.eval -  average_rank: 3.28
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9439    0.9439    0.9439       410
     positive     0.0800    0.0800    0.0800        25

     accuracy                         0.8943       435
    macro avg     0.5120    0.5120    0.5120       435
 weighted avg     0.8943    0.8943    0.8943       435

Evaluating: 100%|█████████████████████████████████| 5/5 [00:11<00:00,  2.35s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
*************************************************

Train epoch 0/0 (Cur. train loss: 8.5517): 100%|█| 17/17 [03:53<00:00, 13.76s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:12<00:00,  2.44s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | TEST SET | AFTER 17 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.9431330251693726
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.903448275862069
INFO - haystack.modeling.evaluation.eval -  f1: 0.16
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5317241379310345
INFO 

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_8"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=5,
    batch_size=6,
    grad_acc_steps=1,
    save_dir=save_dir,
    evaluate_every=3,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=1,
    optimizer_name = "RAdam"
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /|\  /w\   /w\   /w\   /w\   /w\   /w\   /w\   /w\   /w\   /|\  /|\  /w\   /w\   /|\  /|\
INFO - haystack.modeling.data_handler.data_silo -  /'\   /'\   / \   / \   / \   /

INFO - haystack.modeling.evaluation.eval -  average_rank: 2.08
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9208    0.9208    0.9208       265
     positive     0.1600    0.1600    0.1600        25

     accuracy                         0.8552       290
    macro avg     0.5404    0.5404    0.5404       290
 weighted avg     0.8552    0.8552    0.8552       290

Train epoch 0/4 (Cur. train loss: 5.3502):  71%|▋| 12/17 [01:47<00:42,  8.54s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:09<00:00,  1.81s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 12 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.7731596469879151
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8551724137931035
INFO - haystack.modeling.evaluation.eval -  f1: 0.16
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5075862068965518
INFO - haystack.modeling.evaluation.eval -  average_rank: 2.04
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9208    0.9208    0.9208       265
     positive     0.1600    0.1600    0.1600        25

     accuracy                         0.8552       290
    macro avg     0.5404    0.5404    0.5404       290
 weighted avg     0.8552    0.8552    0.8552       290

Train epoch 1/4 (Cur. train loss: 13.7033):  76%|▊| 13/17 [02:01<00:35,  8.81s/i
Evaluating: 100%|█████████████████████████████████|

Train epoch 2/4 (Cur. train loss: 9.6942):  65%|▋| 11/17 [01:39<00:51,  8.59s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:08<00:00,  1.76s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 45 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.6514105606079101
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8620689655172413
INFO - haystack.modeling.evaluation.eval -  f1: 0.20000000000000004
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5310344

INFO - haystack.modeling.evaluation.eval -  f1: 0.28
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5779310344827586
INFO - haystack.modeling.evaluation.eval -  average_rank: 1.6
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9321    0.9321    0.9321       265
     positive     0.2800    0.2800    0.2800        25

     accuracy                         0.8759       290
    macro avg     0.6060    0.6060    0.6060       290
 weighted avg     0.8759    0.8759    0.8759       290

Train epoch 3/4 (Cur. train loss: 3.2988):  71%|▋| 12/17 [01:56<00:43,  8.78s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:08<00:00,  1.75s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 63 BATCHES *****
*****

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 0.5181501913070679
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8758620689655172
INFO - haystack.modeling.evaluation.eval -  f1: 0.28
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5779310344827586
INFO - haystack.modeling.evaluation.eval -  average_rank: 1.48
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9321    0.9321    0.9321       265
     positive     0.2800    0.2800    0.2800        25

     accuracy                         0.8759       290
    macro avg     0.6060    0.6060    0.6060       290
 weighted avg     0.8759    0.8759    0.8759       290

Train epoch 4/4 (Cur. train loss: 3.8331):  76%|▊| 13/17 [02:02<00:35,  8.77s/it
Evaluating: 100%|█████████████████████████████████|

In [6]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_9"

In [7]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=2,
    batch_size=6,
    grad_acc_steps=1,
    save_dir=save_dir,
    evaluate_every=5,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=2,
    optimizer_name = "SGD",
    learning_rate = 1e-6
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /w\   /|\  /w\   /w\   /|\  /w\   /w\   /w\   /|\  /w\   /w\   /w\   /w\   /w\   /w\   /|\
INFO - haystack.modeling.data_handler.data_silo -  / \   /'\   / \   /'\   /'\   

INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.8988505747126436
INFO - haystack.modeling.evaluation.eval -  f1: 0.12
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5094252873563219
INFO - haystack.modeling.evaluation.eval -  average_rank: 2.96
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9463    0.9463    0.9463       410
     positive     0.1200    0.1200    0.1200        25

     accuracy                         0.8989       435
    macro avg     0.5332    0.5332    0.5332       435
 weighted avg     0.8989    0.8989    0.8989       435

Train epoch 0/1 (Cur. train loss: 4.1950): 100%|█| 17/17 [04:21<00:00, 15.36s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:12<00:00,  2.55s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^

In [7]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_10"

In [8]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=3,
    batch_size=6,
    grad_acc_steps=1,
    save_dir=save_dir,
    evaluate_every=4,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=2,
    optimizer_name = "SGD",
    learning_rate = 1e-7
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /w\   /w\   /w\   /w\   /w\   /|\  /w\   /w\   /|\  /w\   /w\   /w\   /|\  /w\   /w\   /|\
INFO - haystack.modeling.data_handler.data_silo -  /'\   / \   /'\   / \   /'\   

INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.903448275862069
INFO - haystack.modeling.evaluation.eval -  f1: 0.16
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5317241379310345
INFO - haystack.modeling.evaluation.eval -  average_rank: 2.52
INFO - haystack.modeling.evaluation.eval -  report: 
                precision    recall  f1-score   support

hard_negative     0.9488    0.9488    0.9488       410
     positive     0.1600    0.1600    0.1600        25

     accuracy                         0.9034       435
    macro avg     0.5544    0.5544    0.5544       435
 weighted avg     0.9034    0.9034    0.9034       435

Train epoch 0/2 (Cur. train loss: 7.9087):  94%|▉| 16/17 [04:11<00:11, 11.28s/it
Evaluating: 100%|█████████████████████████████████| 5/5 [00:13<00:00,  2.66s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^

Train epoch 1/2 (Cur. train loss: 13.3560): 100%|█| 17/17 [03:31<00:00, 12.44s/i
Evaluating: 100%|█████████████████████████████████| 5/5 [00:13<00:00,  2.63s/it]
INFO - haystack.modeling.evaluation.eval -  

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EVALUATION | DEV SET | AFTER 36 BATCHES *****
***************************************************
\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

INFO - haystack.modeling.evaluation.eval -  
 _________ text_similarity _________
INFO - haystack.modeling.evaluation.eval -  loss: 1.0554401969909668
INFO - haystack.modeling.evaluation.eval -  task_name: text_similarity
INFO - haystack.modeling.evaluation.eval -  acc: 0.903448275862069
INFO - haystack.modeling.evaluation.eval -  f1: 0.16
INFO - haystack.modeling.evaluation.eval -  acc_and_f1: 0.5317241379310345
INFO -